In [2]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import LeaveOneGroupOut, GroupKFold,TimeSeriesSplit
from sklearn.model_selection import cross_val_score

from sklearn.pipeline import Pipeline
from feature_engine.datetime import DatetimeFeatures
from feature_engine.creation import CyclicalFeatures

from spatialkfold.blocks import spatial_blocks
import geopandas as gpd


import matplotlib.pyplot as plt
import seaborn as sns

import os,sys,glob,warnings
from tqdm import tqdm

In [3]:
warnings.filterwarnings("ignore", category=UserWarning)

In [4]:
data = pd.read_csv("../results/CSVs/Train_clean.csv", parse_dates=["date"], index_col=["date"])

In [5]:
data.head()

,lat,lng,temperature,precipitation,humidity,global_radiation,hydrometric_level,N,NE,E,...,lc_14,lc_21,lc_22,lc_23,lc_31,lc_32,lc_33,lc_41,lc_51,stn_ID
date,,,,,,,,,,,,,,,,,,,,,
2016-01-01,45.496780,9.257515,1.536364,0.0,84.608392,52.440559,65.918955,0.916667,0.416667,0.891304,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,454909257
2016-01-01,45.171919,9.488997,1.521168,0.0,100.000000,23.218978,14.234551,1.466667,1.628947,1.587500,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,451709488
2016-01-01,45.281956,8.988563,1.829167,0.0,99.972222,23.631944,13.580502,0.929167,1.400000,1.043333,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,452808988
2016-01-01,45.542665,9.205603,1.066434,0.0,94.713287,46.510490,9.671466,0.800000,0.392857,0.409000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,455409205
2016-01-01,45.548517,8.847322,0.511888,0.0,98.909091,28.972028,13.920369,0.759877,0.825000,0.325000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,455408847


# Pollutant Features

In [6]:
# Since measured pollutants are only avaialbe for the station, it makes sence to calcualte the mean of entire stations
# Such values can be used for inference as well
pollutant_feat = data.groupby("date")[["pm10","o3","so2","no2"]].mean()

In [7]:
data.drop(["pm10","o3","so2","no2","pm25_aqi", "pm10_aqi", "no2_aqi", "o3_aqi","so2_aqi", "aqi"],axis=1,inplace=True)

In [8]:
data = pd.merge(data,pollutant_feat,right_index=True,left_index=True)

In [9]:
data.iloc[:,-10:].tail()

,lc_31,lc_32,lc_33,lc_41,lc_51,stn_ID,pm10,o3,so2,no2
date,,,,,,,,,,
2021-12-31,0.0,0.0,0.0,0.0,0.0,452808988,41.146661,2.97524,2.46682,42.488469
2021-12-31,0.0,0.0,0.0,0.0,0.0,455209044,41.146661,2.97524,2.46682,42.488469
2021-12-31,0.0,0.0,0.0,0.0,0.0,455509227,41.146661,2.97524,2.46682,42.488469
2021-12-31,0.0,0.0,0.0,0.0,0.0,454409167,41.146661,2.97524,2.46682,42.488469
2021-12-31,0.0,0.0,0.0,0.0,0.0,453909282,41.146661,2.97524,2.46682,42.488469


# Lag Features

In [10]:
# After investigating the data for daily, weekly, and monthly seasonality, we extracted lag features corresponding to the identified patterns
# Exercise caution when dealing with multiple time series having different timestamps. Lag features for each station should be calculated separately.
lags = [1,3,7,15,30]
for l in lags:
    # Shift the timeseries index to get the lagged versions
    df_shift = data.loc[:, ["lat","lng","temperature", "precipitation", "humidity", "pm10", "o3", "so2", "no2"]].shift(periods=l, freq="D")
    # Join back to the original dataframe
    data = data.merge(df_shift, on=["date", 'lat','lng'], how="left", suffixes=("", f"_lag_{l}"))

# Sanity Check
df = data.copy()
df.sort_values(by=['lat','lng', 'date'],inplace=True)
df.iloc[2860:,:][['lat','lng',"temperature", "temperature_lag_30"]].head(38)

,lat,lng,temperature,temperature_lag_30
date,,,,
2021-12-27,45.171919,9.488997,5.295833,7.161806
2021-12-28,45.171919,9.488997,5.872917,5.240278
2021-12-29,45.171919,9.488997,4.063194,4.447222
2021-12-30,45.171919,9.488997,3.718750,4.624476
2021-12-31,45.171919,9.488997,2.810417,0.145833
2016-01-01,45.281956,8.988563,1.829167,NaN
2016-01-02,45.281956,8.988563,5.954861,NaN
2016-01-03,45.281956,8.988563,9.568056,NaN
2016-01-04,45.281956,8.988563,13.712500,NaN


# Date Features

In [11]:
# Date Faeatures to extract
# Features selected based on observation in EDA for PM2.5
FTs_To_Extract = ["day_of_year", "day_of_week", "week", "weekend", "month", "quarter"]
cyclical_FTs = ["day_of_year", "day_of_week", "week", "month", "quarter"]

pipe = Pipeline([
    
    # create datetime features.
    ('date', DatetimeFeatures(
        variables="index",
        features_to_extract=FTs_To_Extract
    )),

    # apply sine and cosine transformation.
    ('cyclical', CyclicalFeatures(
        variables=cyclical_FTs,
        drop_original=True
    )),
])

# Extract features.
data = pipe.fit_transform(data)

# Sanity Check
dtfs = data.copy()
dtfs.iloc[:,92:].head()

,week_sin,week_cos,month_sin,month_cos,quarter_sin,quarter_cos
date,,,,,,
2016-01-01,-2.449294e-16,1.0,0.5,0.866025,1.0,6.123234e-17
2016-01-01,-2.449294e-16,1.0,0.5,0.866025,1.0,6.123234e-17
2016-01-01,-2.449294e-16,1.0,0.5,0.866025,1.0,6.123234e-17
2016-01-01,-2.449294e-16,1.0,0.5,0.866025,1.0,6.123234e-17
2016-01-01,-2.449294e-16,1.0,0.5,0.866025,1.0,6.123234e-17


In [12]:
data.to_csv("../results/CSVs/02_Train_generated_features.csv", index=True)

# Feature Selection

In [13]:
data.dropna(inplace=True) # Drop NAN generated by Lag features
data.reset_index(inplace=True)

## Spatial Folds

In [13]:
gdf = gpd.GeoDataFrame(data,geometry=gpd.points_from_xy(data.lng, data.lat, crs=4326))

In [14]:
blocks = spatial_blocks(gdf=gdf, width=0.05, height=0.05, 
                                  method='continuous', orientation='tb-lr' ,
                                  nfolds=10, random_state= 175)

In [15]:
stn_block = gpd.overlay (gdf, blocks)

In [16]:
stn_block.head()

,date,lat,lng,temperature,precipitation,humidity,global_radiation,hydrometric_level,N,NE,...,day_of_week_sin,day_of_week_cos,week_sin,week_cos,month_sin,month_cos,quarter_sin,quarter_cos,folds,geometry
0,2016-02-03,45.496780,9.257515,9.820139,11.000000,70.611111,133.208333,66.725220,4.465116,1.722642,...,8.660254e-01,-0.5,0.558647,0.829406,0.866025,0.5,1.0,6.123234e-17,8,POINT (9.25751 45.49678)
1,2016-02-03,45.491633,9.248738,9.452778,11.252672,69.760425,132.626996,68.163121,4.971651,1.945756,...,8.660254e-01,-0.5,0.558647,0.829406,0.866025,0.5,1.0,6.123234e-17,8,POINT (9.24874 45.49163)
2,2016-02-03,45.499584,9.247327,9.478959,11.241143,70.086328,132.577828,66.461595,4.685062,1.862417,...,8.660254e-01,-0.5,0.558647,0.829406,0.866025,0.5,1.0,6.123234e-17,8,POINT (9.24733 45.49958)
3,2016-02-04,45.496780,9.257515,15.334266,0.000000,78.825175,118.853147,67.095767,0.996000,1.746970,...,1.224647e-16,-1.0,0.558647,0.829406,0.866025,0.5,1.0,6.123234e-17,8,POINT (9.25751 45.49678)
4,2016-02-04,45.491633,9.248738,15.229167,0.000000,76.200745,119.606913,68.708333,1.142215,1.794240,...,1.224647e-16,-1.0,0.558647,0.829406,0.866025,0.5,1.0,6.123234e-17,8,POINT (9.24874 45.49163)


# Recursive Feature Elimination

In [17]:
X = stn_block.drop(["pm25","stn_ID","date","lat","lng","folds","geometry"],axis=1)
y = stn_block["pm25"]

sp_folds = stn_block.folds.values.ravel()
tmp_folds = stn_block.date.values.ravel()

In [18]:
X.columns

Index(['temperature', 'precipitation', 'humidity', 'global_radiation',
       'hydrometric_level', 'N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW',
       'dtm_milan', 'aspect', 'dusaf15', 'geologia', 'hillshade', 'ndvi_2019',
       'plan_curvature', 'profile_curvature', 'water_distance', 'slope', 'spi',
       'tri', 'twi', 'geo_0', 'geo_1', 'geo_2', 'geo_3', 'geo_4', 'geo_5',
       'geo_6', 'lc_11', 'lc_12', 'lc_14', 'lc_21', 'lc_22', 'lc_23', 'lc_31',
       'lc_32', 'lc_33', 'lc_41', 'lc_51', 'pm10', 'o3', 'so2', 'no2',
       'temperature_lag_1', 'precipitation_lag_1', 'humidity_lag_1',
       'pm10_lag_1', 'o3_lag_1', 'so2_lag_1', 'no2_lag_1', 'temperature_lag_3',
       'precipitation_lag_3', 'humidity_lag_3', 'pm10_lag_3', 'o3_lag_3',
       'so2_lag_3', 'no2_lag_3', 'temperature_lag_7', 'precipitation_lag_7',
       'humidity_lag_7', 'pm10_lag_7', 'o3_lag_7', 'so2_lag_7', 'no2_lag_7',
       'temperature_lag_15', 'precipitation_lag_15', 'humidity_lag_15',
       'pm10_lag_15', 'o

## Spatial Features

In [19]:
group_cvs =  LeaveOneGroupOut() #initialise splitter 
init_model = RandomForestRegressor(n_jobs=100,random_state=123) # initialise model
features = list(X.columns) # make list with ordered features
tol = 0.0001 # Define the threshold

# build initial model using all the features
r2_full = cross_val_score(init_model, X, y, scoring= 'r2', cv= group_cvs.split(X, y, sp_folds), n_jobs=25, error_score='raise').mean()

# we initialise a list where we will collect the
# features we should remove
features_to_remove = []

# set a counter to know which feature is being evaluated
count = 1

# now we loop over all the features, in order of importance:
# remember that features in the list are ordered
# by importance
for feature in tqdm(features):
    count = count + 1

    # fit model with all variables minus the removed features
    # and the feature to be evaluated and calculate the new r2
    r2_feat = cross_val_score(init_model,X.drop(features_to_remove + [feature], axis=1), y, scoring= 'r2', cv= group_cvs.split(X, y, sp_folds), n_jobs=-1, error_score='raise').mean()


    # determine the drop in the r2
    diff_r2 = r2_full - r2_feat

    # compare the drop in r2 with the tolerance
    # we set previously
    if diff_r2 < tol:
        # if the drop in the r2 is small and we remove the
        # feature, we need to set the new r2 to the one based on
        # the remaining features
        r2_full = r2_feat
        
        # and append the feature to remove to the collecting list
        features_to_remove.append(feature)

print('total features to remove: ', len(features_to_remove))

# determine the features to keep (those we won't remove)
features_to_keep = [x for x in features if x not in features_to_remove]
print('total features to keep: ', len(features_to_keep))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 94/94 [32:53<00:00, 21.00s/it]

total features to remove:  85
total features to keep:  9


In [20]:
print(f"Spatial Features which should be kept: {features_to_keep}")

Spatial Features which should be kept: ['global_radiation', 'hydrometric_level', 'no2', 'no2_lag_1', 'temperature_lag_15', 'o3_lag_15', 'so2_lag_15', 'temperature_lag_30', 'o3_lag_30']


In [21]:
print(f"The R² with the selected features is: {r2_full:.2f}")

The R² with the selected features is: 0.99


## Temporal Features

In [22]:
group_cvs = TimeSeriesSplit(n_splits=10) #initialise splitter 
init_model = RandomForestRegressor(n_jobs=100,random_state=123) # initialise model
features = list(X.columns) # make list with ordered features
tol = 0.0001 # Define the threshold

# build initial model using all the features
r2_full = cross_val_score(init_model, X, y, scoring= 'r2', cv= group_cvs.split(X, y, tmp_folds), n_jobs=25, error_score='raise').mean()

# we initialise a list where we will collect the
# features we should remove
features_to_remove = []

# set a counter to know which feature is being evaluated
count = 1

# now we loop over all the features, in order of importance:
# remember that features in the list are ordered
# by importance
for feature in tqdm(features):
    count = count + 1

    # fit model with all variables minus the removed features
    # and the feature to be evaluated and calculate the new r2
    r2_feat = cross_val_score(init_model,X.drop(features_to_remove + [feature], axis=1), y, scoring= 'r2', cv= group_cvs.split(X, y, tmp_folds), n_jobs=10, error_score='raise').mean()


    # determine the drop in the r2
    diff_r2 = r2_full - r2_feat

    # compare the drop in r2 with the tolerance
    # we set previously
    if diff_r2 < tol:
        # if the drop in the r2 is small and we remove the
        # feature, we need to set the new r2 to the one based on
        # the remaining features
        r2_full = r2_feat
        
        # and append the feature to remove to the collecting list
        features_to_remove.append(feature)

print('total features to remove: ', len(features_to_remove))

# determine the features to keep (those we won't remove)
features_to_keep = [x for x in features if x not in features_to_remove]
print('total features to keep: ', len(features_to_keep))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 94/94 [56:15<00:00, 35.91s/it]

total features to remove:  31
total features to keep:  63


In [23]:
print(f"Temporal Features which should be kept: {features_to_keep}")

Temporal Features which should be kept: ['precipitation', 'humidity', 'hydrometric_level', 'N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'dtm_milan', 'aspect', 'dusaf15', 'hillshade', 'ndvi_2019', 'plan_curvature', 'water_distance', 'slope', 'twi', 'geo_3', 'geo_5', 'geo_6', 'lc_12', 'lc_22', 'lc_23', 'lc_31', 'lc_32', 'lc_33', 'lc_41', 'lc_51', 'pm10', 'so2', 'no2', 'humidity_lag_1', 'so2_lag_1', 'no2_lag_1', 'temperature_lag_3', 'pm10_lag_3', 'o3_lag_3', 'so2_lag_3', 'temperature_lag_7', 'precipitation_lag_7', 'humidity_lag_7', 'o3_lag_7', 'precipitation_lag_15', 'humidity_lag_15', 'so2_lag_15', 'no2_lag_15', 'temperature_lag_30', 'precipitation_lag_30', 'humidity_lag_30', 'o3_lag_30', 'so2_lag_30', 'weekend', 'day_of_year_sin', 'day_of_week_sin', 'day_of_week_cos', 'week_sin', 'week_cos', 'month_sin', 'month_cos', 'quarter_sin', 'quarter_cos']


In [24]:
print(f"The R² with the selected features is: {r2_full:.2f}")

The R² with the selected features is: 0.93
